In [ ]:
from IPython.display import HTML

import json
try:
    import json
except:
    %pip install json
    import json

try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

from operator import itemgetter

import os

from datetime import datetime

try:
    from sortedcontainers import SortedDict

except:
    %pip install sortedcontainers
    from sortedcontainers import SortedDict

try:
    import plotly.express as px
except:
    %pip install plotly.express
    import plotly.express as px



Load data file - events_all.json

In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

    print(number_ex_by_year)

## Histogram - number of exhibitions by year - all

In [ ]:

x_values = number_ex_by_year.keys()
y_values = number_ex_by_year.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Year", "y": "Number of exhibitions"},
title="Number of exhibitions by year"
)
fig.show()

# History - exhibitions per year - non MoMa

In [ ]:
filepath = "api"
filename = "events_nonmoma.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

    print(number_ex_by_year)

    x_values = number_ex_by_year.keys()
    y_values = number_ex_by_year.values()

    fig = px.bar(x=x_values, y=y_values, 
    labels={"x": "Year", "y": "Number of exhibitions"},
    title="Number of exhibitions by year - non-MoMa data"
    )
    fig.show()

# A histogram of number of exhibitions per institution !(MoMA+PS1+whitney) 

In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_by_org = {}

    data = json.load(json_file)
    for event in data.get("events"):
        carried_out_by = event.get("carried_out_by")
        for org in carried_out_by: 
            if "Person" in org:
                continue
           

            # get label for id
            org_file = "../data/group/" + org.split("/").pop() + ".json"
            with open(os.path.join(org_file), 'r') as org_file:
                orgdata = json.load(org_file)

                org_label = orgdata.get("_label")

            if org_label in number_by_org:
                number_by_org[org_label] += 1
            else:
                number_by_org[org_label] = 1
        
    number_by_org = SortedDict(number_by_org)

    
    # sort by number
    number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

print(number_by_org)


In [ ]:
x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by org"
)
fig.show()

# remove museum of modern art and moma PS1
 

In [ ]:
if "The Museum of Modern Art" in number_by_org:
    del number_by_org["The Museum of Modern Art"]
if "MoMA PS1" in number_by_org:
    del number_by_org["MoMA PS1"]

In [ ]:
# sort by number
number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by org"
)
fig.show()


# Number of exhibitions per venue

In [ ]:
filepath = "api"
filename = "events_all.json"



with open(os.path.join(filepath, filename), 'r') as json_file:
    events_venue = {}
    number_dict = {}

    data = json.load(json_file)
    for event in data.get("events"):
        venue = event.get("venue")

        
        
        if venue in number_dict:
            number_dict[venue] += 1
            events_venue[venue].append(event)
        else:
            number_dict[venue] = 1
            events_venue[venue] = [event]
        
    number_dict = SortedDict(number_dict)

    
    # sort by number
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    

    for venue in events_venue:
        
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        with open('data/summary/venue/exhibitions/' + venue_filename + '.json', 'w') as ex_file:
            ex_file.write(json.dumps({"meta": {"venue": venue, "exhibition_count": number_dict[venue]},"events": events_venue[venue]}))


    venue_index = []

    for venue in number_dict:
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        venue_index.append({"name": venue, "filename": venue_filename,  "number_exhibitions": number_dict[venue]})

    with open('data/summary/venue/exhibitions/index.json', 'w') as ex_file:
            ex_file.write(json.dumps({"venues": venue_index}))

    

In [ ]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

In [ ]:
if "MoMA" in number_dict:
    del number_dict["MoMA"]
if "MoMA PS1" in number_dict:
    del number_dict["MoMA PS1"]

In [ ]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

# Number of artists per Nationality

In [ ]:
filepath = "api"
filename = "persons_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_dict = {}

    data = json.load(json_file)
    for event in data.get("persons"):

        if "nationality" in event:
            nat = event.get("nationality")

            if nat == "":
                continue
            if nat == "American":
                continue
            if nat in number_dict:
                number_dict[nat] += 1
            else:
                number_dict[nat] = 1
        
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    # sort by number
    
    print(number_dict)

In [ ]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Nationality", "y": "Number of people"},
title="Number of people involved in exhibitions by nationality (excluding Americans and empty values)"
)
fig.show()

# Number of exhibitions per year for each organisation

In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")
        start = event.get("start")
        year = start.split("-")[0]

        if org in number_ex_by_year:

            if year in number_ex_by_year[org]:
                number_ex_by_year[org][year] += 1
            else:
                number_ex_by_year[org][year] = 1
        else:
            number_ex_by_year[org] = {}
            number_ex_by_year[org][year] = 1
        
    #number_ex_by_year = SortedDict(number_ex_by_year)

    

    for org in number_ex_by_year:

        numbers = number_ex_by_year[org]
        numbers = SortedDict(numbers)
        print(numbers)
        x_values = numbers.keys()
        y_values = numbers.values()

        fig = px.bar(x=x_values, y=y_values, 
        labels={"x": "Year", "y": "Number of exhibitions"},
        title="Number of exhibitions by year - " + org
        )
        fig.show()
        

# keyword frequency in exhibition title - all


In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    numbers = {}

    data = json.load(json_file)
    
    for event in data.get("events"):
        
        title = event.get("_label")
       
        keywords = title.split(" ")
        for keyword in keywords:
            if keyword in numbers:
                numbers[keyword] += 1
            else:
                numbers[keyword] = 1
    
    numbers = dict(sorted(numbers.items(), key=lambda item: item[1]))
    
        
        
        
    print(numbers)
    
    
        

# keyword frequency in exhibition title - non MoMa

In [ ]:
filepath = "api"
filename = "events_nonmoma.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    numbers = {}

    data = json.load(json_file)
    
    for event in data.get("events"):
        
        title = event.get("_label")
       
        keywords = title.split(" ")
        for keyword in keywords:
            if keyword in numbers:
                numbers[keyword] += 1
            else:
                numbers[keyword] = 1
    
    numbers = dict(sorted(numbers.items(), key=lambda item: item[1]))
    
        
        
        
    print(numbers)
    
    
        

# Create a forced network diagram

diagram with links between artists and organisations (via inclusion in exhibitions)

In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        if org == "The ":
            continue
        if org == "MoMA PS1":
            continue
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

          
        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            #links_sources  = [x['source'] for x in links]

            links.append({"source": org, "target": person, "value": 1})

    with open('org_artist_links_nonmoma.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))
        

In [ ]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        list_orgs = ['Tanager Gallery', "Brata Gallery", "Hansa Gallery", "MoMA PS1"]
        if org not in list_orgs:
            continue
        
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            links.append({"source": org, "target": person, "value": 1})


    list_links_persons  = [x['target'] for x in links]
    list_nodes_ids = [x['id'] for x in nodes]

    nonmoma_links = []
    moma_links = []

    for link in links:
        if link.get("source") not in ["MoMA PS1", "The Museum of Modern Art"]:
            nonmoma_links.append(link)
        else:
            moma_links.append(link)

    list_persons_nonmoma = [x['target'] for x in nonmoma_links]
    list_persons_moma = [x['target'] for x in moma_links]

    # get persons in moma list that are not in nonmoma list and then remove links including them 
    # and remove corresponding nodes

    list_persons_uniquetomoma= []
    for person in list_persons_moma:
        if person not in list_persons_nonmoma:
            list_persons_uniquetomoma.append(person)

    
    i = 0
    while i < len(links):
        link = links[i]
        if "target" in link: 
            person = link.get("target")
            if person in list_persons_uniquetomoma:
                
                del links[i]
        i += 1

    i = 0
 

    with open('org_artist_links_10thstreet.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))


 
        

Timeline visualisation

In [ ]:
# Make JSON data file

filepath = "api"
filename = "events_all.json"

events_list = []

with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start").split("T")[0]

        start_year = start.split("-")[0]
        start_month = start.split("-")[1]
        start_day = start.split("-")[2]

        end = event.get("start").split("T")[0]

        end_year = start.split("-")[0]
        end_month = start.split("-")[1]
        end_day = start.split("-")[2]
         
        

        display_date = str(start) + " - " + str(end)

        headline = event.get("_label")

        event_vis = {
      "media": {
        "url": "",
        "caption": "",
        "credit": "",
        "thumbnail": ""
      },
      "display_date": display_date,
      "start_date": {
        "year": start_year,
        "month": start_month,
        "day": start_day
      },
      "end_date": {
        "year": end_year,
        "month": end_month,
        "day": end_day
      },
      "text": {
        "headline": headline,
        "text": ""
      },
      "type": "title",
      "background": {  "color": "#3f83e8"  }
    }
        events_list.append(event_vis)


  


with open('events_all_timeline.json', 'w') as file:
        file.write(json.dumps({"events": events_list}))

In [ ]:
# Make JSON data file

filepath = "data/summary/venue/exhibitions"
filename = "index.json"



with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)

    for venue in data.get("venues"):
      events_list = []

      filename_venue = venue.get("filename") + ".json"
      with open(os.path.join(filepath, filename_venue), 'r') as venue_file:
        data = json.load(venue_file)
     

        for event in data.get("events"):

          start = event.get("start").split("T")[0]
          start_year = start.split("-")[0]
          start_month = start.split("-")[1]
          start_day = start.split("-")[2]

          end = event.get("start").split("T")[0]
          end_year = start.split("-")[0]
          end_month = start.split("-")[1]
          end_day = start.split("-")[2]
          display_date = str(start) + " - " + str(end)
          headline = event.get("_label")

          event_vis = {
              "media": {   "url": "",   "caption": "",   "credit": "",  "thumbnail": ""   },
              "display_date": display_date,
              "start_date": {   "year": start_year,  "month": start_month, "day": start_day },
              "end_date": { "year": end_year, "month": end_month, "day": end_day},
              "text": {   "headline": headline,"text": ""  },
              "type": "title",
              "background": {  "color": "#3f83e8"  }
            }
          events_list.append(event_vis)

      
      with open('data/summary/venue/timeline/' + filename_venue , 'w') as file:
        file.write(json.dumps({"events": events_list}))

In [ ]:
from IPython.display import HTML

import json
try:
    import json
except:
    %pip install json
    import json



from operator import itemgetter

import os

from datetime import datetime


local_html_file = '<head>\
</head>\
<body>\
        <link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/>\
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>\
            <h1>Timeline of exhibition data</h1>\
        <div id="timeline-embed" style="width: 100%; height: 400px"></div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed", "./data/summary/venue/timeline/HansaGallery.json");</script>\
</body>'

HTML(local_html_file)

In [28]:
try:
    import ipywidgets
except:
    %pip install ipywidgets
    import ipywidgets

from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML, Image
from IPython.core.display import Javascript        
 
import os

try:
    import json
except:
    %pip install json
    import json 



    
selectOptions = []
selectOptions = [('Please select a venue', '')]

filepath = "data/summary/venue/exhibitions"
filename = "index.json"
filepath_timeline = "data/summary/venue/timeline"



with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)

    for venue in data.get("venues"):
        filename = venue.get("filename")
        name = venue.get("name")
        selectOptions.append( (name , filename))

local_html_file = ""

def dropdown_eventhandler(change):
    
    filename_venue=change.new
    with open(os.path.join(filepath_timeline, filename_venue + '.json'), 'r') as venue_file:
        local_html_file = '<link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/><script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>'

        local_html_file += '<h1>Timeline of exhibition data ' + filename_venue + '</h1>\
        <div id="timeline-embed-' + venue.get("filename") + '" style="background-color:blue;width: 100%; height: 700px">' + filename_venue +'</div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed-' + venue.get("filename") + '", "./data/summary/venue/timeline/' + filename_venue + '.json");</script>'
        display(HTML(local_html_file))     
    
   

selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')

display(selectObject)





Dropdown(options=(('Please select a venue', ''), ('42nd Venice Biennale', '42ndVeniceBiennale'), ('44th Venice…